In [ ]:
!pip install py7zr

In [ ]:
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U transformers==4.38.0
!pip install evaluate rouge-score nltk

In [ ]:
pip install -q -U datasets==2.16.1

In [ ]:
# from huggingface_hub import notebook_login

# notebook_login()

In [1]:
from transformers.utils import send_example_telemetry

send_example_telemetry("summarization_notebook", framework="pytorch")

In [2]:
model_checkpoint = "Mr-Vicky-01/Bart-Finetuned-conversational-summarization"

In [48]:
from datasets import load_dataset
from evaluate import load

raw_datasets = load_dataset("samsum")
metric = load("rouge")

Generating train split: 0 examples [00:00, ? examples/s]

/opt/conda/lib/python3.10/site-packages/datasets/download/streaming_download_manager.py:778: FutureWarning: The 'verbose' keyword in pd.read_csv is deprecated and will be removed in a future version.
  return pd.read_csv(xopen(filepath_or_buffer, "rb", download_config=download_config), **kwargs)


In [49]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['File_path', 'Articles', 'Summaries'],
        num_rows: 2224
    })
})

In [50]:
raw_datasets["train"][0]

{'File_path': 'politics',
 'Articles': 'Budget to set scene for election..Gordon Brown will seek to put the economy at the centre of Labour\'s bid for a third term in power when he delivers his ninth Budget at 1230 GMT. He is expected to stress the importance of continued economic stability, with low unemployment and interest rates. The chancellor is expected to freeze petrol duty and raise the stamp duty threshold from £60,000. But the Conservatives and Lib Dems insist voters face higher taxes and more means-testing under Labour...Treasury officials have said there will not be a pre-election giveaway, but Mr Brown is thought to have about £2bn to spare...- Increase in the stamp duty threshold from £60,000. - A freeze on petrol duty. - An extension of tax credit scheme for poorer families. - Possible help for pensioners The stamp duty threshold rise is intended to help first time buyers - a likely theme of all three of the main parties\' general election manifestos. Ten years ago, buye

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

In [7]:
prefix = ""

In [8]:
max_input_length = 1024
max_target_length = 64

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["dialogue"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [9]:
preprocess_function(raw_datasets['train'][:1])

{'input_ids': [[0, 10127, 5219, 35, 38, 17241, 1437, 15269, 4, 1832, 47, 236, 103, 116, 50121, 50118, 39237, 35, 9136, 328, 50121, 50118, 10127, 5219, 35, 38, 581, 836, 47, 3859, 48433, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[0, 10127, 5219, 17241, 15269, 8, 40, 836, 6509, 103, 3859, 4, 2]]}

In [10]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [11]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

config.json:   0%|          | 0.00/1.67k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/328 [00:00<?, ?B/s]

In [12]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [13]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [14]:
training_args = Seq2SeqTrainingArguments(
    output_dir=f"{model_checkpoint.split('/')[-1]}-finetuned-xsum",
    evaluation_strategy="epoch",
    learning_rate=2e-5,  # Adjust learning rate
    per_device_train_batch_size=4,  # Adjust batch size
    per_device_eval_batch_size=2,
    weight_decay=0.01,  # Adjust weight decay
    adam_epsilon=1e-8,
    gradient_accumulation_steps=8,  # Adjust gradient accumulation steps
    warmup_steps=2000,  # Adjust warmup steps
    save_total_limit=2,
    num_train_epochs=2,  # Adjust number of training epochs
    logging_steps=20,
    predict_with_generate=True,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [15]:
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
0,0.977000,1.570660,42.038900,21.359000,32.190800,38.736900,59.880200
1,0.988200,1.408053,41.902400,21.788000,32.400500,38.841200,59.751800


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


TrainOutput(global_step=920, training_loss=1.010591332031333, metrics={'train_runtime': 3506.6212, 'train_samples_per_second': 8.402, 'train_steps_per_second': 0.262, 'total_flos': 1.7688796613935104e+16, 'train_loss': 1.010591332031333, 'epoch': 2.0})

In [16]:
trainer.model.save_pretrained("/kaggle/working/conversation_summarization_modelv1")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


In [17]:
model2 = AutoModelForSeq2SeqLM.from_pretrained("/kaggle/working/conversation_summarization_modelv1")

In [26]:
import torch

# Check if GPU is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [27]:
def generate_summary_old(text):
    inputs = tokenizer([text], max_length=1024, return_tensors='pt', truncation=True).to(device)
    summary_ids = model.generate(inputs['input_ids'], max_new_tokens=100, do_sample=False)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [32]:
def generate_summary(text):
    inputs = tokenizer([text], max_length=1024, return_tensors='pt', truncation=True)
    summary_ids = model2.generate(inputs['input_ids'], max_new_tokens=100, do_sample=False)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [46]:
text_to_summarize = """The rise of artificial intelligence (AI) has brought significant advancements across various industries, including healthcare, finance, transportation, and entertainment. AI technologies, such as machine learning, natural language processing, and computer vision, have enabled automation of tasks, personalized recommendations, predictive analytics, and autonomous vehicles. However, the increasing integration of AI raises ethical concerns regarding data privacy, algorithmic bias, job displacement, and potential misuse of AI-powered systems. Despite these challenges, the potential benefits of AI for enhancing efficiency, decision-making, and innovation are driving continued research and development in the field."""

In [ ]:
# i = 110
# text_to_summarize = raw_datasets["test"]['dialogue'][i]
summary = generate_summary_old(text_to_summarize)
print("OLD:\n",summary)
summary1 = generate_summary(text_to_summarize)
print("\n\nNEW:\n",summary1)

In [ ]:
raw_datasets["test"]['summary'][i]

In [ ]:
tokenizer.save_pretrained("/kaggle/working/tokenizer")